# Machine learning on pulse fit parameters

This notebook demonstrates the functions in `fastepy` to perform machine learning on the pulse best-fit parameters in order to separate signal from __intrinsic__ background. The functionality relies mostly in `sklearn`.

We start with the relevant imports.

In [1]:
import numpy as np
import logging
import time
from pathlib import PosixPath
from fastespy.io.readpydata import convert_data_to_ML_format, load_data_rikhav
from fastespy.mlscikit import hyperpartune
from fastespy.utils import init_logging

Now we define some parameters of the analysis, e.g., where we save the data, where we want to put the output files (for plotting them later), and which classifier we want to use. 

The following classifiers from `sklearn` are currently implemented:

- `dt`: Decision trees. This is mostly for testing as the training is fast. 
- `bdt`: Boosted descision trees.
- `rf`: Random forests
- `mlp`: multi-layer perceptron. 

In [2]:
# input directory with data
indir = "/Users/manuelmeyer/Documents/TESData/August2020/"   # August 2020 data

# classifier with naming scheme as above
classifier_name = 'dt'

# number of cores to use
n_jobs = 8

# random seed for the analysis 
random_state = 42

# Use a coarse grid for the hyper parameter search?
coarse_grid = True

# Use class weights in hyper parameter optimization?
class_weight_grid = False

# log transform the data?
log_data = True

# Use PCA to transform data?
use_pca = True

# The number of k-folds for inner and outer loop
kfolds = 5

# output directory for ML results
outdir = f"/Users/manuelmeyer/Documents/TESData/ml_results_clf-{classifier_name}_log-data{log_data:n}_pca{use_pca:n}/"
print(outdir)

/Users/manuelmeyer/Documents/TESData/ml_results_clf-dt_log-data1_pca1/


Instead of running the rest of the notebook, you could also run the analysis on the command line by executing the following cell:

In [3]:
#%run ../fastespy/scripts/scikit_hyperpartuning.py -i $indir -o $outdir -c $classifier_name --n_jobs $n_jobs 
# --random_state $random-state --coarse_grid $coarse_grid --class-weight-grid $class_weight_grid --log-data $log_data --use-pca $use_pca

Initialize nice colorful logging

In [4]:
init_logging("INFO", color=True)

Get the input files and set up output dir

In [5]:
in_dir = PosixPath(indir)
out_dir = PosixPath(outdir)

if not out_dir.exists():
    out_dir.mkdir(parents=True)

Get a list of files:

In [6]:
files = list(in_dir.glob("*.npy"))

logging.info("Using files:")
for f in files:
    logging.info(f)

1735487905.py:   3 --- INFO: Using files:
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GHzGBWP-intrinsics-50MHz-20mV-1day-16-fit000.npy
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GHzGBWP-intrinsics-50MHz-20mV-72hrs-3-fit000.npy
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GHz-light-000-fit.npy
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GBWP-intrinsics-50MHz-20mV-1day-9-fit000.npy
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GHzGBWP-intrinsics-50MHz-20mV-1day-13-fit000.npy
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GHzGBWP-intrinsics-50MHz-20mV-1day-14-fit000.npy
1735487905.py:   5 --- INFO: /Users/manuelmeyer/Documents/TESData/August2020/0.3RN-1.5GHzGBWP-intrinsics-50MHz-20mV-72hrs-1-fit000.npy
1735487905.py:   5 --- INFO: /Users/manuelme

Next, we define the names of the features we would like to use. These are of course specific to the formatting of the output files. I will here assume the file formats used in Rikhav's analysis

In [7]:
# define the feature names
feature_names = []

# these names will be removed
remove = ['data', 'time', 'pulse integral raw', 'voltage error',
          'error', 'start time in hrs', 'end time in hrs',
          'trigger time'
          ]

# read in one file to get the feature names
# and remove those you don't want
x = np.load(files[0], allow_pickle=True).tolist()
for k in x[1].keys():
    if not k in remove and not 'error' in k:
        feature_names.append(k)

logging.info("Using features names {}".format(feature_names))

2405857269.py:  17 --- INFO: Using features names ['rise time', 'decay time', 'constant', 'amplitude', 'pulse height', 'chi2 reduced', 'pulse integral fit']


We now define some minimal data cleaning cuts: 

In [8]:
# define light cleaning cuts
light_cleaning_cuts = {
    "chi2 reduced": "chi2 < 6.",
    "decay time": "decay < 10.e-6",
    "trigger time": "(trigger >= 29.5e-6) & (trigger <= 30.8e-6)",  # from gaussian fit, 5 sigma interval
}
logging.info("Using cleaning cuts {}".format(light_cleaning_cuts))

838698770.py:   7 --- INFO: Using cleaning cuts {'chi2 reduced': 'chi2 < 6.', 'decay time': 'decay < 10.e-6', 'trigger time': '(trigger >= 29.5e-6) & (trigger <= 30.8e-6)'}


Next we read the output from Rikhav's fits. The `result` dict contains the best fit parameters. The `data` dict contains the raw time lines. 
The `t_tot_hrs` gives the total time of the data run in hours. 

In [9]:
# read the data
result, data, t_tot_hrs = load_data_rikhav(files, feature_names, light_cleaning_cuts=light_cleaning_cuts)

print (result.keys())
print (data.keys())
print (t_tot_hrs)

readpydata.py: 291 --- INFO: Reading data
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  3.40it/s]
readpydata.py: 339 --- INFO: Rejected 1561 triggers in light file
readpydata.py: 350 --- INFO: In total, there are 1066 light events and 39580 background events for an observation time of 517.91 hours


dict_keys(['type', 'rise time', 'decay time', 'constant', 'amplitude', 'pulse height', 'chi2 reduced', 'pulse integral fit'])
dict_keys(['time', 'data'])
517.906026640339


The data needs to be converted into the standard `sklearn` format: `X` is an array with shape `[n_samples, n_features]` and `y` contains the class labels. 

In [10]:
X, y = convert_data_to_ML_format(result,
                                 feature_names,
                                 bkg_type=0,  # this specifies which result['type'] corresponds to background
                                 signal_type=1  # this specifies which result['type'] corresponds to signal
                                )
print(X.shape)
print(y.shape)

(40646, 7)
(40646,)


Lastly, we set up the default parameters and the hyper-parameter grid. Every combination of the hyper-parameter grid will be tested. These grids are pre-defined but you could also define your own. 

In [11]:
# default parameters
default_pars = hyperpartune.default_pars[classifier_name]
print(default_pars)

# parameter grid
if coarse_grid:
    param_grid = hyperpartune.param_grid_coarse[classifier_name]
else:
    param_grid = hyperpartune.param_grid[classifier_name]
print(param_grid)

# the classifier
classifier = hyperpartune.clf[classifier_name](random_state=random_state,
                                               **default_pars)

{'criterion': 'gini', 'min_samples_leaf': 1}
{'ccp_alpha': array([0.     , 0.00025, 0.0005 , 0.00075, 0.001  ]), 'min_samples_split': array([ 2, 12, 22, 32, 42, 52]), 'max_depth': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20])}


This last step will run the hyper-parameter optimization for `kfolds x kfolds` iterations. The results will be written to an `npy` file in the output directory. We will look at the results in another notebook. 

In [ ]:
t0 = time.time()
hyperpartune.run_hyper_par_opt_all_folds(X, y,
                                         feature_names=feature_names,
                                         classifier=classifier,
                                         default_pars=default_pars,
                                         param_grid=param_grid,
                                         t_tot_hrs=t_tot_hrs,
                                         data=data,
                                         kfolds=kfolds,
                                         classifier_name=classifier_name,
                                         random_state=random_state,
                                         log_data=log_data,
                                         use_pca=use_pca,
                                         out_dir=out_dir,
                                         n_jobs=n_jobs)

print("It took {0:.2f} s".format(time.time() - t0))

hyperpartune.py:1118 --- INFO: Transforming data to log space
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/hyperpartune.py:406: RuntimeWarning: divide by zero encountered in log10
  X[:, j] = np.log10(-X[:, j])
hyperpartune.py:1127 --- INFO: Running optimization for split 1 / 5
hyperpartune.py:1010 --- INFO: Transforming data using PCA


Discarded 234 triggers in log transform
Fitting 5 folds for each of 570 candidates, totalling 2850 fits
The parameter search took 42.56 s
Profiling over parameters
Refitting on whole test data set and computing learning curve and confusion matrix
[learning_curve] Training set sizes: [ 3232  6465  9698 12931 16164 19397 22630 25863]


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/manuelmeyer/Python/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:696: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/manuelmeyer/Python/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 334, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/Users/manuelmeyer/Python/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 53, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'DecisionTreeClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/manuelmeyer/Python/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn

[learning_curve] Training set sizes: [ 3232  6465  9698 12931 16164 19397 22630 25863]


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  40 out of  40 | elapsed:    0.4s finished


...

[learning_curve] Training set sizes: [ 3233  6466  9699 12932 16165 19398 22631 25864]


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/hyperpartune.py:222: RuntimeWarning: invalid value encountered in true_divide
  tp_rate = np.sum(y * y_pred) / float(np.sum(y))
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/hyperpartune.py:222: RuntimeWarning: invalid value encountered in true_divide
  tp_rate = np.sum(y * y_pred) / float(np.sum(y))
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/hyperpartune.py:222: RuntimeWarning: invalid value encountered in true_divide
  tp_rate = np.sum(y * y_pred) / float(np.sum(y))
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/hyperpartune.py:222: RuntimeWarning: invalid value encountered in true_divide
  tp_rate = np.sum(y * y_pred) / float(np.sum(y))
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/hyperpartune.py:222: RuntimeWarning: invalid value encountered in true_divide
  tp_rate = np.sum(y * y_pred) / float(np.sum(y))
[Parallel(n_jobs=8)]

==== AUC : bkg_pred ====
train: 2.279e-05
test: 3.218e-05
==== AUC : tp_efficiency ====
train: 9.637e-01
test: 9.671e-01
==== AUC : score ====
train: 9.971e-01
test: 9.992e-01
==== AUC : Significance ====
train: 3.412
test: 2.974
==== Accuracy : bkg_pred ====
train: 1.006e-05
test: 1.877e-05
==== Accuracy : tp_efficiency ====
train: 1.000e+00
test: 9.812e-01
==== Accuracy : score ====
train: 9.995e-01
test: 9.986e-01
==== Accuracy : Significance ====
train: 4.734
test: 3.739
==== Significance : bkg_pred ====
train: 4.023e-06
test: 1.073e-05
==== Significance : tp_efficiency ====
train: 1.000e+00
test: 9.718e-01
==== Significance : score ====
train: 6.116e+00
test: 4.527e+00
==== Significance : Significance ====
train: 6.116
test: 4.527


/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/feldman_cousins.py:311: UserWarning: The given `mus`-array is probably to coarse. No upper limit found for `n_obs` = 8. Setting the upper limit to inf.
  warnings.warn(('The given `mus`-array is probably to coarse.' +
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/feldman_cousins.py:311: UserWarning: The given `mus`-array is probably to coarse. No upper limit found for `n_obs` = 9. Setting the upper limit to inf.
  warnings.warn(('The given `mus`-array is probably to coarse.' +
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/feldman_cousins.py:311: UserWarning: The given `mus`-array is probably to coarse. No upper limit found for `n_obs` = 10. Setting the upper limit to inf.
  warnings.warn(('The given `mus`-array is probably to coarse.' +
/Users/manuelmeyer/Python/fastespy/fastespy/mlscikit/feldman_cousins.py:311: UserWarning: The given `mus`-array is probably to coarse. No upper limit found for `n_obs` = 11. Setting the 

Fitting 5 folds for each of 570 candidates, totalling 2850 fits
